## Visualisierung und Vorhersage der Coronavirus-Ausbreitung
<!-- (CC BY 4.0) Gert Herold, 2022 -->


In diesem Skript werden anhand täglich gemeldeter Infektionszahlen der aktuelle Stand der Coronavirus-Infektionslage visualisiert sowie der zukünftige Verlauf geschätzt.
Die Schätzungen basieren auf Modellannahmen zum Wachstum der Infiziertenzahl.

Import benötigter Pakete:

In [ ]:
#%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

#### Aktuelle Daten laden

Quellen für Daten zum Verlauf der Covid-Pandemie gibt es mittlerweile sehr viele. Unter anderem stellen die [WHO](https://covid19.who.int) und das ["European Centre for Disease Prevention and Control"](https://www.ecdc.europa.eu/en/covid-19/data) Datensätze zum Download zur Verfügung.
Wir verwenden den sehr detaillierten Datensatz der Seite "[Our World in Data](https://ourworldindata.org/coronavirus-source-data)".
Die Daten laden wir direkt aus dem Internet und legen damit ein [DataFrame-Objekt aus dem pandas-Modul](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) an (aktuell knapp 43 MB):

In [ ]:
df = pd.read_csv("https://github.com/owid/covid-19-data/raw/master/public/data/owid-covid-data.csv", encoding='latin1', on_bad_lines='skip') # wird aktuell gehalten

# Falls der obige Aufruf nicht funktioniert, können Sie die Datei auch unter
# https://covid.ourworldindata.org/data/owid-covid-data.csv
# herunterladen, ins aktuelle Verzeichnis kopieren und dann so öffnen:
# df = pd.read_csv("owid-covid-data.csv", encoding='latin1', error_bad_lines=False)

# Für eine grobe Übersicht eine Teil-Ausgabe der Daten:
df 

Ein DataFrame ist in etwa ein Dictionary, das jedoch noch wesentlich mehr Funktionen mitbringt. Wir geben mal aus, wie die Spalten (Keys), auf die zugegriffen werden kann, heißen:

In [ ]:
print(df.keys())

Wir definieren einige Aliase:

In [ ]:
key_country = 'location'# Key für die Länder
key_date = 'date' # Key fürs Tagesdatum
date_format = '%Y-%m-%d' # Format des Datums zum späteren Parsen
default_key = 'total_cases' # Standardkey für Daten, die geplottet werden sollen

Jetzt erzeugen wir aus der Länder-Spalte ein Set (unsortiert), um Duplikate zu vermeiden, und sortieren die Liste alphabetisch. 
Dann geben wir aus, wie viele und welche Länder in diesem Datensatz enthalten sind.

In [ ]:
sorted_list = np.sort(list(set(df[key_country])))
nlaender = len(sorted_list)
print('Anzahl Länder:',nlaender)
print('Länderliste:',sorted_list)

Ausgabe aller Ländernamen, die die Zeichenkette 'United' enthalten:

In [ ]:
[_ for _ in set(df[key_country]) if 'United' in _]

Schauen wir uns ein paar Daten für Deutschland an:

In [ ]:
df[df[key_country] == 'Germany']

### Lineare Regression:

In erster Näherung kann davon ausgegangen werden, dass der Anstieg der Anzahl an Infizierten exponentiell ist, also der Rechenvorschrift
$$
y=a\cdot \mathrm{e}^{b \cdot x}
$$
folgt. Dabei ist $y$ die Infektionszahl und $x$ der Tag. Die Koeffizienten $a$ und $b$ sind zunächst unbekannt.
Das Problem ist in dieser Formulierung nicht linear, aber kann einfach linear gemacht werden, indem der Logarithmus auf beiden Seiten gebildet wird:

$$
\log y= \log (a\cdot \mathrm{e}^{b \cdot x}) = \log a + b\cdot x
$$

Wie in Hausaufgabe 4 kann nun ein Regressionsgerade berechnet werden.
Hier wird dafür die Funktion [*numpy.polyfit()*](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) verwendet, die zusätzlich auch eine Gewichtung der Eingangsdaten erlaubt.
Je geringer die Anzahl an Infektionen, desto größer kann der relative Fehler angenommen werden. Deshalb werden hier kleine Zahlen auch geringer gewichtet.

Das tatsächliche Wachstum hängt von einer Vielzahl von Faktoren hat. 
Zusätzlich zum exponentiellen Wachstum ist in diesem Skript exemplarisch das Modell eines polynomialen Wachstums (mit nur einem unbekannten Exponenten) implementiert:
$$
y=a\cdot x^b
$$
bzw. als lineare Formulierung:
$$
\log y=\log (a\cdot x^b) = \log a + b\cdot \log x
$$

Da das Wachstum in der Realität (glücklicherweise) beschränkt ist, wird irgendwann eine Sättigung erreicht (dann, wenn ein bestimmter Anteil der Bevölkerung infiziert ist). 
Ein solcher Verlauf könnte mithilfe der [logistischen Funktion](https://de.wikipedia.org/wiki/Logistische_Funktion) modelliert werden.
Diese lässt sich jedoch nicht mehr so einfach linear abbilden.


Wir basteln uns eine Klasse, die die Daten für ein Land aufbereiten kann.

In [ ]:
np.random.seed(1)
# Liste der verfügbaren Farben erstellen
colorlist = list(matplotlib.colors.TABLEAU_COLORS.values()) + list(matplotlib.colors.XKCD_COLORS.values())

class covid_trend():
    """
    Klasse zum Laden, Berechnen und Plotten länderspezifischer Daten und Trends
    """    
    def __init__( self, country):
        self.country = country
        # eindeutige Farbe zuordnen (pop entfernt Element aus Liste)
        self.color = colorlist.pop(0)
        # alle Daten für das Land holen
        self.data = df[df[key_country] == country]
        # eindeutige Linienstärke zuordnen
        self.lw = np.random.uniform(1.3,4.3)
    
    def __get_idays(self):
        # Tage im Datensatz als Liste von Zahlen
        return [(datetime.strptime(_, date_format)-datetime(1970,1,1)).days for _ in self.data[key_date]]
        
    def __get_dates(self):
        # Tage als Liste von Datumsangaben
        return [datetime.strptime(_, date_format) for _ in self.data[key_date]]#[::-1]
    
    def numbers(self, key=default_key, sum_all=False):
        """
        Gib Anzahl der Fälle zurück.
        Parameter:
            key: Welche Zahlen sollen geholt werden (Default: Gesamtzahl)
            sum_all: Wenn "True", berechne kumulative Summe der Größe
        """
        cases = np.array(self.data[key])
        #cases[np.isnan(cases)] = 0
        # falls gewünscht, berechne kumulative Summe
        if sum_all: 
            cases[np.isnan(cases)] = 0
            cases = np.cumsum(cases)
        return cases

    
    def plot_data(self, mask_limit=0, key=default_key, sum_all=False):
        """
        Kurve mit "Messdaten" plotten
        Parameter:
            mask_limit: Anzahl Fälle, ab der Kurve geplottet werden soll
            key: Plotte Anzahl Infektionen (key_new_cases) oder Verstorbener (key_new_deaths)
            sum_all: Wenn "True", plotte kumulative Summe der Größe
        """
        # Hole die Daten
        data = self.numbers(key, sum_all)
        # Index, ab dem geplottet werden soll, festlegen
        ind_start = np.argmax(data>=mask_limit)
        # eigentliche Kurve plotten
        plt.plot(self.dates[ind_start:],data[ind_start:],
                 ls='-', lw = self.lw,
                 c=self.color,
                 label="Data %s"%self.country, alpha=0.9)
        # x-Achse mit Datumsangaben versehen
        plt.gca().format_xdata = mdates.DateFormatter('%Y-%m-%d')
    
   
    def params(self, x, y, model='exp'):
        # Funktion, die die lineare Regression durchführt und die Parameter a und b zurückgibt.
        if model == 'pol':
            x = np.log(x)
        logy = np.log(y)
        # nimm nur gültige Werte für die Regression
        valid_inds = np.isfinite(x) & np.isfinite(logy)
        # Polyfit mit Grad 1 = Lineare Regression
        yvalid=y[valid_inds]
        ba = np.polyfit(x[valid_inds], logy[valid_inds], deg=1, w=(yvalid/yvalid.max())**0.5)
        return np.exp(ba[1]), ba[0]
    
    
    def plot_trend(self, end_date='', model='exp', days_past=21, key=default_key, sum_all=False):
        """ 
        Funktion, die die Daten fittet und ggf. Vorhersage plottet sowie den letzten Wert zurück gibt
        Parameter:
            end_date: Vorhersagedatum im Format YYYY-MM-DD; Default: heute
            model: exponentiell oder polynomiales Wachstum
            days_past: Anzahl der Tage (genauer: Datumsangaben im Datensatz) in der Vergangenheit, die für die Modellierung noch mitbetrachtet werden sollen
            key: Betrachte Anzahl Infektionen (key_new_cases) oder Verstorbener (key_new_deaths)
            sum_all: Wenn "True", plotte kumulative Summe der Größe
        Returns:
            letzter Wert der Vorhersage, Exponent
        """
        
        if end_date=='':
            # letzter Tag, der uns interessiert
            endday = self.idays[-1]
        else:
            endday = (datetime.fromisoformat(end_date)-datetime(1970,1,1)).days
        
        # "x-Werte", für die mit dem verwendeten Modell y-Werte berechnet werden sollen
        days = np.linspace(self.idays[-int(1.2*days_past)],endday,100)
        daysdate = np.array([datetime.fromtimestamp(_*3600*24) for _ in days])
        
        # Damit die Exponenten nicht so hoch werden (verursacht numerische Probleme), 
        # setzen wir den Referenztag auf ca. einen Monat vor unseren Modelldaten
        day0 = self.idays[-days_past-1]-30
        
        # Hole gewünschte Zahl an Datumsangaben aus dem Datensatz (Annahme: jeder Tag hat einen Eintrag), Referenz auf ersten Tag
        xdata = np.array(self.idays[(-days_past-1):])-day0
        # Hole gewünschte Zahl an Daten aus dem Datensatz (Annahme: jeder Tag hat einen Eintrag)
        ydata = self.numbers(key=key)[(-days_past-1):]
        
        a,b = self.params(xdata, ydata, model=model)
        # Plotstil anpassen, je nach Modell
        if model == 'exp':
            infected_trend = a*np.exp(b*(days-day0))
            ls='--'
        else:
            infected_trend = a*(days-day0)**b
            ls = ':'
         
        # Trendkurve plotten
        plt.plot(daysdate, infected_trend, 
                 ls=ls, lw = self.lw,
                 c=self.color, 
                 label="Trend %s, %s" % (self.country,'exp'), alpha=0.7)
        # x-Achse mit Datumsangaben versehen
        plt.gca().format_xdata = mdates.DateFormatter('%Y-%m-%d')
        
        # zusätzlich Vorhersage für letzten betrachteten Tag zurückgeben
        return infected_trend[-1],b
            
    
    idays = property(__get_idays)
    #: Attribut, dass die Datumsangaben in "Tage seit 1.1.1970" enthält
    
    dates = property(__get_dates)
    #: Attribut, dass die Datumsangaben als Liste von datetime-Objekten enthält

Wir instanzieren ein Dictionary mit Objekten der obigen Klasse für verschiedene Länder:

In [ ]:
laenderliste = ["Germany","United Kingdom",
                "Poland", "Spain", "China", 
                "United States", "Brazil", 
                "Austria", "Israel"]

# Erstelle Objekte für obige Länderliste in Dictionary
land = {name:covid_trend(country=name) for name in laenderliste}
land

### Visualisierung der Datenlage

Nun erstellen wir Plots der vorhandenen Daten (gesamte Zahl an gemeldeten Erkrankungen und Verstorbenen). 
Da der Anstieg zunächst stark nichtlinear ist, plotten wir die Daten mit logarithmisch skalierter y-Achse.
Außerdem geben wir die Daten für Deutschland als Text aus.

In [ ]:
# Plotte Anzahl der Gesamtinfektionen
plt.figure(1, figsize=(20,8))

for einland in land.values():
    einland.plot_data(mask_limit=1e4)

plt.ylabel('Number of overall infections')
plt.legend()

plt.grid(True)
plt.yscale("log")
plt.show()

name_key = "Germany"
print('Infections %s:' % name_key,land[name_key].dates[-1],land[name_key].numbers()[-1])

# Plotte Anzahl der Verstorbenen
plt.figure(2, figsize=(20,8))

for name in land.values():
    name.plot_data(key='total_deaths',mask_limit=300)

plt.ylabel('Number of deaths')
plt.legend()

plt.grid(True)
plt.yscale("log")

plt.show()

print('Deaths %s:' % name_key,land[name_key].dates[-1],land[name_key].numbers('total_deaths')[-1])

Um ein Gefühl für das aktuelle Infektionsgeschehen zu erhalten, schauen wir uns auch mal die geglätteten Werte der Neu-Infektionen relativ zur Einwohnerzahl an:

In [ ]:
plt.figure(1, figsize=(20,13))

for einland in land.values():
    einland.plot_data(mask_limit=0, key='new_cases_smoothed_per_million')

plt.ylabel('Number of NEW infections per 1e6')
plt.legend()

plt.grid(True)
plt.yscale("log")
plt.ylim([1e-2,2e4])
plt.show()



### Vorhersage

Wir wollen zu den Plots der Gesamtinfektionen Trendlinien hinzufügen. Wir berechnen diese aus den Daten der letzten 14 Tage.
Außerdem geben wir die hochgerechnete Anzahl an Infizierten und Verstorbenen zu einem von uns gewählten Datum aus.

In [ ]:
auswahl = ["Germany", "Israel",  "Brazil", "United States", "Spain"]

# Hier geben wir das zukünftige Datum ein, das uns interessiert
datum='2022-08-14'
tage = 14


plt.figure(3, figsize=(22,15))

#########################################################
limit=5000
plt.subplot(221)
for name in auswahl:
    land[name].plot_data(mask_limit=limit)
    nexp,bexp=land[name].plot_trend(end_date=datum, days_past=tage)
    print('%s (Exp. Anstieg: %9.2e) -> Anzahl Infektionen %s: %d'  %(name, bexp, datum, nexp))
    npol,bpol=land[name].plot_trend(end_date=datum, days_past=tage, model='pol')
    print('%s (Pol. Anstieg: %9.2e) -> Anzahl Infektionen %s: %d\n'%(name, bpol, datum, npol))


#plt.legend()
plt.yscale("log")
plt.ylim([limit,None])
plt.ylabel('Number of infections')
plt.grid(True)

#########################################################
limit=5000
plt.subplot(222)

for name in auswahl:
    land[name].plot_data(mask_limit=limit, key='total_deaths')
    nexp,bexp=land[name].plot_trend(end_date=datum, days_past=tage, key='total_deaths')
    print('%s (Exp. Anstieg: %9.2e) -> Anzahl Verstorben %s: %d'  %(name, bexp, datum, nexp))
    npol,bpol=land[name].plot_trend(end_date=datum, days_past=tage, model='pol', key='total_deaths')
    print('%s (Pol. Anstieg: %9.2e) -> Anzahl Verstorben %s: %d\n'%(name, bpol, datum, npol))

plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
plt.yscale("log")
plt.ylim([limit,None])
plt.ylabel('Number of deaths')
plt.grid(True)


#########################################################
# Betrachtung bezogen auf die Bevölkerungsgröße
#######################################################pc#
limit=5000
plt.subplot(223)

for name in auswahl:
    land[name].plot_data(mask_limit=limit, key='total_cases_per_million')
    nexp,bexp=land[name].plot_trend(end_date=datum, days_past=tage, key='total_cases_per_million')
    npol,bpol=land[name].plot_trend(end_date=datum, days_past=tage, model='pol', key='total_cases_per_million')

#plt.legend()
plt.yscale("log")
plt.ylabel("Number of infections per 1e6")
plt.grid(True)

#########################################################
limit=100
plt.subplot(224)

for name in auswahl:
    land[name].plot_data(mask_limit=limit, key='total_deaths_per_million')
    nexp,bexp=land[name].plot_trend(end_date=datum, days_past=tage, key='total_deaths_per_million')
    npol,bpol=land[name].plot_trend(end_date=datum, days_past=tage, model='pol', key='total_deaths_per_million')


#plt.legend()
plt.yscale("log")
plt.ylabel("Number of deaths per 1e6")

plt.grid(True)

plt.show()

**Man beachte, dass ein exponentielles Wachstum nur dann so weiterläuft wie zuvor, wenn noch keine Sättigung aufgetreten ist und die Infektionstätigkeit nicht durch zusätzliche Maßnahmen oder andere äußere Umstände verändert wird.**
Das heißt, der angezeigte Trend kann sowohl zu optimistisch als auch zu pessimistisch sein. Es ist jedoch aus dem vergangenen Verlauf ersichtlich, dass es nur wenige "plötzliche" Veränderungen eines Trends gibt.